Identificación de créditos riesgosos usando SVM
===

Las entidades financieras desean mejorar sus procedimientos de aprobación de créditos con el fin de disminuir los riesgos de no pago de la deuda, lo que acarrea pérdidas a la entidad. El problema real consiste en poder decidir si se aprueba o no un crédito particular con base en información que puede ser fácilmente recolectada por teléfono o en la web. Se tiene una muestra de 1000 observaciones. Cada registro contiene 20 atributos que recopilan información tanto sobre el crédito como sobre la salud financiera del solicitante. Construya un sistema de recomendación que use máquinas de vectores de soporte.

El archivo de datos se encuentra disponible en el siguiente link:

https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/credit.csv



Los atributos y sus valores son los siguientes:

     Attribute 1:  (qualitative)
     	      Status of existing checking account
     	      A11 :      ... <    0 DM
     	      A12 : 0 <= ... <  200 DM
     	      A13 :      ... >= 200 DM /
     	            salary assignments for at least 1 year
     	      A14 : no checking account

     Attribute 2:  (numerical)
     	      Duration in month

     Attribute 3:  (qualitative)
     	      Credit history
     	      A30 : no credits taken/
     	            all credits paid back duly
     	      A31 : all credits at this bank paid back duly
     	      A32 : existing credits paid back duly till now
     	      A33 : delay in paying off in the past
     	      A34 : critical account/
     	            other credits existing (not at this bank)

     Attribute 4:  (qualitative)
     	      Purpose
     	      A40 : car (new)
     	      A41 : car (used)
     	      A42 : furniture/equipment
     	      A43 : radio/television
     	      A44 : domestic appliances
     	      A45 : repairs
     	      A46 : education
     	      A47 : (vacation - does not exist?)
     	      A48 : retraining
     	      A49 : business
     	      A410 : others

     Attribute 5:  (numerical)
     	      Credit amount

     Attribute 6:  (qualitative)
     	      Savings account/bonds
     	      A61 :          ... <  100 DM
     	      A62 :   100 <= ... <  500 DM
     	      A63 :   500 <= ... < 1000 DM
     	      A64 :          .. >= 1000 DM
     	      A65 :   unknown/ no savings account

     Attribute 7:  (qualitative)
     	      Present employment since
     	      A71 : unemployed
     	      A72 :       ... < 1 year
     	      A73 : 1  <= ... < 4 years  
     	      A74 : 4  <= ... < 7 years
     	      A75 :       .. >= 7 years

     Attribute 8:  (numerical)
     	      Installment rate in percentage of disposable income

     Attribute 9:  (qualitative)
     	      Personal status and sex
     	      A91 : male   : divorced/separated
     	      A92 : female : divorced/separated/married
     	      A93 : male   : single
     	      A94 : male   : married/widowed
     	      A95 : female : single

     Attribute 10: (qualitative)
     	      Other debtors / guarantors
     	      A101 : none
     	      A102 : co-applicant
     	      A103 : guarantor

     Attribute 11: (numerical)
     	      Present residence since

     Attribute 12: (qualitative)
     	      Property
     	      A121 : real estate
     	      A122 : if not A121 : building society savings agreement/
     				   life insurance
     	      A123 : if not A121/A122 : car or other, not in attribute 6
     	      A124 : unknown / no property

     Attribute 13: (numerical)
     	      Age in years

     Attribute 14: (qualitative)
     	      Other installment plans 
     	      A141 : bank
     	      A142 : stores
     	      A143 : none

     Attribute 15: (qualitative)
     	      Housing
     	      A151 : rent
     	      A152 : own
     	      A153 : for free

     Attribute 16: (numerical)
              Number of existing credits at this bank

     Attribute 17: (qualitative)
     	      Job
     	      A171 : unemployed/ unskilled  - non-resident
     	      A172 : unskilled - resident
     	      A173 : skilled employee / official
     	      A174 : management/ self-employed/
     		         highly qualified employee/ officer

     Attribute 18: (numerical)
     	      Number of people being liable to provide maintenance for

     Attribute 19: (qualitative)
     	      Telephone
     	      A191 : none
     	      A192 : yes, registered under the customers name

     Attribute 20: (qualitative)
     	      foreign worker
     	      A201 : yes
     	      A202 : no


In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC

df = pd.read_csv(
    "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/german.csv"
)

df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,property,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job
0,< 0 DM,6,critical,radio/tv,1169,unknown,> 7 yrs,4,single male,none,...,real estate,67,none,own,2,1,1,yes,yes,skilled employee
1,1 - 200 DM,48,repaid,radio/tv,5951,< 100 DM,1 - 4 yrs,2,female,none,...,real estate,22,none,own,1,2,1,none,yes,skilled employee
2,unknown,12,critical,education,2096,< 100 DM,4 - 7 yrs,2,single male,none,...,real estate,49,none,own,1,1,2,none,yes,unskilled resident
3,< 0 DM,42,repaid,furniture,7882,< 100 DM,4 - 7 yrs,2,single male,guarantor,...,building society savings,45,none,for free,1,1,2,none,yes,skilled employee
4,< 0 DM,24,delayed,car (new),4870,< 100 DM,1 - 4 yrs,3,single male,none,...,unknown/none,53,none,for free,2,2,2,none,yes,skilled employee


In [ ]:
#
# Use el transformador LabelEncoder para preprocesar
# las columnas alfanuméricas del dataframe.
#
# Use los primeros 900 datos para entrenamiento del
# modelo y los 100 restantes para validación.
# 
# Construya el SVM usando los valores por defecto de
# los parámetros.
#
# Compute la matriz de confusión para la muestra de
# validación.
#
# Rta/
# True
# True
# True
# True
#
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np

df = pd.read_csv(
    "https://raw.githubusercontent.com/jdvelasq/datalabs/master/datasets/german.csv"
)

# >>> Inserte su codigo aquí >>>
objList = df.select_dtypes(include = "object").columns
le = LabelEncoder()
for i in objList:
    df[i] = le.fit_transform(df[i].astype(str))

X = np.array(df.drop(['default'],1))
y = np.array(df['default'])

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=13)

svm = SVC()

svm.fit(X_train,y_train)
svm_y_pred = svm.predict(X_test)
svm_y_pred

cm = confusion_matrix(y_test, svm_y_pred)

# ---->>> Evaluación ---->>>
# cm es la matriz de confusion
print(cm[0][0] == 67)
print(cm[0][1] == 1)
print(cm[1][0] == 30)
print(cm[1][1] == 2)

True
True
True
True


In [ ]:
#
# Encuentre la mejor combinación de kernel y parámetros
# de regularización para los valores suministrados 
# durante el entrenamiento y compute la matriz de 
# confusión para la muestra de prueba.
#
# Rta/
# True
# True
# True
# True
#

kernels = ['rbf', 'linear', 'poly', 'sigmoid']
Cs = [1, 2, 3, 4, 5]

# >>> Inserte su codigo aquí >>>
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=5)

#escalar = StandardScaler()
#X_train = escalar.fit_transform(X_train)
#X_test = escalar.transform(X_test)

param_grid = {'C': Cs,
              'kernel': kernels}

gs_svm = GridSearchCV(SVC(),param_grid,cv=2,n_jobs=-1,verbose=10).fit(X_train,y_train)
svm_y_pred = gs_svm.predict(X_test)
cm = confusion_matrix(y_test, svm_y_pred)
# ---->>> Evaluación ---->>>
# cm es la matriz de confusion
print(cm[0][0] == 68)
print(cm[0][1] == 0)
print(cm[1][0] == 30)
print(cm[1][1] == 2)

Fitting 2 folds for each of 20 candidates, totalling 40 fits
True
True
True
True


array([[68,  0],
       [30,  2]])